In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from preprocess import transform_features, encode_features
import pickle

In [ ]:
import zipfile
with zipfile.ZipFile("/titanic/titanic.zip","r") as zip_ref:
    zip_ref.extractall("/tmp/titanic/")

In [ ]:
train = pd.read_csv("/tmp/titanic/train.csv")
test = pd.read_csv("/tmp/titanic/test.csv")

In [ ]:
train = transform_features(train)
test = transform_features(test)
train_vis = train.copy(deep=True) # data for visualization
train, test, encoders = encode_features(train, test)
train.head()

In [ ]:
# save encoders for later use in transformer
with open("encoders.pkl", "wb") as f:
    pickle.dump (encoders, f)

In [ ]:
X_all = train.drop(['Survived', 'PassengerId'], axis=1)
y_all = train['Survived']

num_test = 0.20
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=23)

In [ ]:
# Choose the type of classifier. 
clf = RandomForestClassifier()

# Choose some parameter combinations to try
parameters = {'n_estimators': [4, 6, 9], 
              'max_features': ['log2', 'sqrt','auto'], 
              'criterion': ['entropy', 'gini'],
              'max_depth': [2, 3, 5, 10], 
              'min_samples_split': [2, 3, 5],
              'min_samples_leaf': [1,5,8]
             }

# Type of scoring used to compare parameter combinations
acc_scorer = make_scorer(accuracy_score)

# Run the grid search
grid_obj = GridSearchCV(clf, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the clf to the best combination of parameters
clf = grid_obj.best_estimator_

# Fit the best algorithm to the data. 
clf.fit(X_train, y_train)

In [ ]:
predictions = clf.predict(X_test)
print(accuracy_score(y_test, predictions))